In [1]:
import re
import pandas as pd
import numpy as np
import string
import subprocess

In [2]:
movie_files = 'The_Shawshank_Redemption.mkv'

input_file = "videos/" + movie_files
output_file = "front_center.flac"

# Get the stream index of the lossless English audio track
cmd = f"ffprobe -v error -select_streams a -show_entries stream=index:stream_tags=language -of csv=p=0 {input_file}"
output = subprocess.check_output(cmd, shell=True).decode("utf-8").strip()
stream_index = None
for line in output.split("\n"):
    index, language = line.split(",")
    if language == "eng":
        stream_index = int(index)
        print(line)
        break

# Extract the center channel of the lossless English audio track
cmd = f"ffmpeg -y -i {input_file} -map 0:a:{stream_index} -ac 1 {output_file}"
subprocess.run(cmd, shell=True)

videos/The_Shawshank_Redemption.mkv: No such file or directory


CalledProcessError: Command 'ffprobe -v error -select_streams a -show_entries stream=index:stream_tags=language -of csv=p=0 videos/The_Shawshank_Redemption.mkv' returned non-zero exit status 1.

In [7]:
# Define the terminal command as a list of strings
whisperx_command = [
    'whisperx',
    'front_center.flac',
    '--highlight_words', 'True',
    '--model', 'large-v3',
    '--language', 'en'
]

# Run the whisperx command
subprocess.run(whisperx_command, check=True)

torchvision is not available - cannot save figures
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.0. Bad things might happen unless you revert torch to 1.x.
>>Performing transcription...
>>Performing alignment...
Failed to align segment (" ¶¶"): no characters in this segment found in model dictionary, resorting to original...
Failed to align segment (" ¶¶"): no characters in this segment found in model dictionary, resorting to original...


CompletedProcess(args=['whisperx', 'front_center.flac', '--highlight_words', 'True', '--model', 'large-v3', '--language', 'en'], returncode=0)

In [8]:
filename = 'srts/The_Shawshank_Redemption.srt'
edlname = 'edls/The_Shawshank_Redemption.edl'

In [9]:
def read_four_lines(filename):
    with open(filename, 'r') as file:
        while True:
            # Read 4 lines from the file
            lines = [file.readline().strip() for _ in range(4)]

            # If there are no more lines left, break the loop
            if not lines[0]:
                break

            yield lines

# Example usage:

text_groups = read_four_lines(filename)
profanity_df = pd.read_csv('dict/profanity.csv',dtype_backend='pyarrow')
profanity_df = profanity_df.apply(lambda col: col.str.lower())
flat_profanity = profanity_df.values.flatten()
flat_profanity = flat_profanity[~pd.isna(flat_profanity)]

In [10]:
# Translation table for removing punctuation
translator = str.maketrans('', '', string.punctuation)
with open(edlname, 'w') as file:
    for text in text_groups:
        word_between_u = re.search(r'<u>(.*?)</u>', text[2])
        if word_between_u:
            word = word_between_u.group(1).lower().translate(translator)
            if word in flat_profanity:
                # Given string
                time_string = text[1]

                # Using regular expression to extract start and stop times
                start_time, stop_time = re.findall(r'(\d+:\d+:\d+,\d+)', time_string)

                # Converting start and stop times to desired format
                start_time = start_time.replace(',', '.')
                stop_time = stop_time.replace(',', '.')
                command = start_time + ' ' + stop_time + ' ' + '1' + ' #' + word
                file.write(command+'\n')